In [1]:
from flask import Flask
app= Flask(__name__)

# Spring Controller와 동일 역할 수행
@app.route("/")
def hello():
    return "<h1>hello python web</h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [2]:
# 1. 카카오 디벨로퍼 페이지 로그인
# 2. 내 애플리케이션 -> 동의항목 -> 닉네임 :필수 / 카카오계정: 선택 / 카톡메시지 : 선택
# 3. 카카오로그인 (https://developers.kakao.com/console/app/640894/product/login) : 활성화 On
# 4. Redirect URL : https://localhost.com 로 설정
# 5. 플랫폼 (https://developers.kakao.com/console/app/640894/config/platform) : Web -> https://www.naver.com https://localhost.com   2개 등록
# 6. kakao api 문서 위치 - https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api
# 7. 주어진 정보 확인
#     GET /oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code HTTP/1.1
#     Host: kauth.kakao.com
#     요약 정보에 있는 Rest API키가  client_id={REST_API_KEY}에 ㄷ배치되면댐
#     Redirect Uri 경우앞서 등록했던 localhost.com
#     sample : 
#     https://kauth.kakao.com/oauth/authorize?response_type=code&client_id={api key}&redirect_uri=https://localhost.com&respomse_type=code
# 9. 해당 url로 요청하면 kakao api를 활용할 수 있는 토큰을 발급받을 수 있고
#    주어진 시간내에 처리하지 않으면 토큰이 만료되므로 먼저 코드를 작성하도록 한다.

 


In [3]:
import requests

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "6f3edf2bef3f028498f9baa6277890e9",
    "redirect_uri" : "https://localhost.com",
    "code" : "z4mTJQ1g5dE6sAGcUVPF-29BRwyVgG6x-zJ00E7QEnUdSDe16LgOEk7Xrx3bMCX1bPclxgopb1UAAAF8MNF1ZA"
}


response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

Error!


In [ ]:
import json
import requests
import datetime
import os

# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"

# 저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# Access Token 갱신을 위한 Refresh
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [ ]:
# Kakao Talk Message 보내기
# 1) https://developers.kakao.com/docs/latest/ko/message/rest-api

import json
import requests

# 토큰 정보 활용
tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

# 메시지 보내기 URL
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# requests parameter 설정
headers = {
    # Authorization: Bearer {ACCESS_TOKEN}
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "지금 집에 도둑이 들어왔어요. 당신의 물품이 다 털려츱니다. 가까운 경찰서에 찾아가세요.",
        "link" : {
            # 이동할 링크 정보니까 사이트 주소를 걸면 더 좋음
            "web_url" : "www.naver.com"
        }
    })
}

# 메시지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print("Error!")
else:
    print("카톡 메시지가 간드아!!!")

In [ ]:
from flask import Flask, url_for, request
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>")
def get_member(id):
    return 'member : ' + id

@app.route("/test", methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        id = request.form['id']
        email = request.form['email']
        
        if not id and not email:
            return add_entity(request.form)
    else:
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()

In [ ]:
# Cors 문제 해결
# pip install flask_cors
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>")
def get_member(id):
    return 'member : ' + id

@app.route("/login", methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        print("POST 처리")
        #userId = request.form['userId']
        #pw = request.form['password']
        #print("userId = {0}, pw = {1}".format(userId, pw))
        # request.get_json() 상대편에서 보낸 json 내용을 읽음
        data = request.get_json()
        print(data)
        
        #return "userId = " + userId + ", password = " + pw
        # json 데이터를 전송
        return jsonify(data)
    else:
        print("GET 처리")
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()